In [1]:
# import needed libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Read the data file
telecom = pd.read_csv("train.csv")
telecom.head()

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,...,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
0,0,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,31.277,87.009,...,0,0,NaN,NaN,NaN,1958,0.0,0.0,0.0,0
1,1,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,0.000,122.787,...,0,0,NaN,1.0,NaN,710,0.0,0.0,0.0,0
2,2,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,60.806,103.176,...,0,0,NaN,NaN,NaN,882,0.0,0.0,0.0,0
3,3,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,156.362,205.260,...,0,0,NaN,NaN,NaN,982,0.0,0.0,0.0,0
4,4,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,240.708,128.191,...,1,0,1.0,1.0,1.0,647,0.0,0.0,0.0,0


In [3]:
#Check for data dimension
telecom.shape

(69999, 172)

In [4]:
telecom.info(verbose=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69999 entries, 0 to 69998
Data columns (total 172 columns):
 #    Column                    Dtype  
---   ------                    -----  
 0    id                        int64  
 1    circle_id                 int64  
 2    loc_og_t2o_mou            float64
 3    std_og_t2o_mou            float64
 4    loc_ic_t2o_mou            float64
 5    last_date_of_month_6      object 
 6    last_date_of_month_7      object 
 7    last_date_of_month_8      object 
 8    arpu_6                    float64
 9    arpu_7                    float64
 10   arpu_8                    float64
 11   onnet_mou_6               float64
 12   onnet_mou_7               float64
 13   onnet_mou_8               float64
 14   offnet_mou_6              float64
 15   offnet_mou_7              float64
 16   offnet_mou_8              float64
 17   roam_ic_mou_6             float64
 18   roam_ic_mou_7             float64
 19   roam_ic_mou_8             float64
 20   roam

Original data file consists of 69999 rows and 172 columns. As per the results there are both numerical and categorical columns

## Handling missing values

In [5]:
#Checking for further missing values per column
null_value_series = telecom.isnull().sum()/len(telecom.index)*100

#Listing columns having more than 5% missing values
null_value_series = null_value_series[null_value_series > 5]
null_value_series

onnet_mou_8                  5.290076
offnet_mou_8                 5.290076
roam_ic_mou_8                5.290076
roam_og_mou_8                5.290076
loc_og_t2t_mou_8             5.290076
loc_og_t2m_mou_8             5.290076
loc_og_t2f_mou_8             5.290076
loc_og_t2c_mou_8             5.290076
loc_og_mou_8                 5.290076
std_og_t2t_mou_8             5.290076
std_og_t2m_mou_8             5.290076
std_og_t2f_mou_8             5.290076
std_og_t2c_mou_8             5.290076
std_og_mou_8                 5.290076
isd_og_mou_8                 5.290076
spl_og_mou_8                 5.290076
og_others_8                  5.290076
loc_ic_t2t_mou_8             5.290076
loc_ic_t2m_mou_8             5.290076
loc_ic_t2f_mou_8             5.290076
loc_ic_mou_8                 5.290076
std_ic_t2t_mou_8             5.290076
std_ic_t2m_mou_8             5.290076
std_ic_t2f_mou_8             5.290076
std_ic_t2o_mou_8             5.290076
std_ic_mou_8                 5.290076
spl_ic_mou_8

#### There are many columns having > 70 % missing values but zero columns are having missing values

### In context of business its important to keep values for rechange columns and hence imputing zero for all rechange columns

In [6]:
recharge_columns = ['total_rech_data_6', 'total_rech_data_7', 'total_rech_data_8',
                 'max_rech_data_6', 'max_rech_data_7', 'max_rech_data_8',
                 'av_rech_amt_data_6', 'av_rech_amt_data_7', 'av_rech_amt_data_8'
                 ]

In [7]:
# impute missing values with 0
telecom[recharge_columns] = telecom[recharge_columns].apply(lambda x: x.fillna(0))

In [8]:
# now, let's make sure values are imputed correctly
print("Missing value ratio:\n")
print(telecom[recharge_columns].isnull().sum()*100/telecom.shape[1])

# summary
print("\n\nSummary statistics\n")
print(telecom[recharge_columns].describe(include='all'))

Missing value ratio:

total_rech_data_6     0.0
total_rech_data_7     0.0
total_rech_data_8     0.0
max_rech_data_6       0.0
max_rech_data_7       0.0
max_rech_data_8       0.0
av_rech_amt_data_6    0.0
av_rech_amt_data_7    0.0
av_rech_amt_data_8    0.0
dtype: float64


Summary statistics

       total_rech_data_6  total_rech_data_7  total_rech_data_8  \
count       69999.000000       69999.000000       69999.000000   
mean            0.619309           0.683981           0.697867   
std             1.762010           1.943193           1.973434   
min             0.000000           0.000000           0.000000   
25%             0.000000           0.000000           0.000000   
50%             0.000000           0.000000           0.000000   
75%             1.000000           1.000000           1.000000   
max            61.000000          54.000000          60.000000   

       max_rech_data_6  max_rech_data_7  max_rech_data_8  av_rech_amt_data_6  \
count     69999.000000     69999

### Checking for columns having single value across all rows

In [9]:
#Checking for columns having majority of single values for the dataset and percentage of single value in each column
telecom.nunique().sort_values().head(25)

std_ic_t2o_mou_6         1
circle_id                1
loc_og_t2o_mou           1
std_og_t2o_mou           1
loc_ic_t2o_mou           1
last_date_of_month_6     1
last_date_of_month_7     1
last_date_of_month_8     1
std_ic_t2o_mou_8         1
std_og_t2c_mou_8         1
std_og_t2c_mou_7         1
std_og_t2c_mou_6         1
std_ic_t2o_mou_7         1
night_pck_user_6         2
night_pck_user_7         2
night_pck_user_8         2
churn_probability        2
fb_user_7                2
fb_user_8                2
fb_user_6                2
monthly_2g_6             5
monthly_2g_8             6
monthly_2g_7             6
monthly_3g_6            10
monthly_3g_8            12
dtype: int64

#### Many columns are having single values present for all the rows and hence dropping these columns as these columns won't be helpful in determining Chrun probability

In [10]:
#Create Data frame for Features and value counts
telecom_count_df = pd.DataFrame(telecom.nunique().sort_values()).reset_index()
telecom_count_df.columns = ['Features','Value_count']

#Get name of columns having Value_count = 1
single_value_col = telecom_count_df[telecom_count_df['Value_count'] == 1]['Features'].tolist()
single_value_col

['std_ic_t2o_mou_6',
 'circle_id',
 'loc_og_t2o_mou',
 'std_og_t2o_mou',
 'loc_ic_t2o_mou',
 'last_date_of_month_6',
 'last_date_of_month_7',
 'last_date_of_month_8',
 'std_ic_t2o_mou_8',
 'std_og_t2c_mou_8',
 'std_og_t2c_mou_7',
 'std_og_t2c_mou_6',
 'std_ic_t2o_mou_7']

In [11]:
print("Shape before dropping single value columns : ", telecom.shape)
#Delete single value columns
telecom = telecom.drop(single_value_col,axis=1)
print("Shape after dropping single value columns : ", telecom.shape)

Shape before dropping single value columns :  (69999, 172)
Shape after dropping single value columns :  (69999, 159)


### Check the distribution of values for columns having 2 unique values

In [12]:
telecom.night_pck_user_6.value_counts(normalize=True)*100


0.0    97.472678
1.0     2.527322
Name: night_pck_user_6, dtype: float64

In [13]:
telecom.night_pck_user_7.value_counts(normalize=True)*100

0.0    97.593059
1.0     2.406941
Name: night_pck_user_7, dtype: float64

In [14]:
telecom.night_pck_user_8.value_counts(normalize=True)*100

0.0    97.898681
1.0     2.101319
Name: night_pck_user_8, dtype: float64

In [15]:
telecom.fb_user_6.value_counts(normalize=True)*100

1.0    91.632514
0.0     8.367486
Name: fb_user_6, dtype: float64

In [16]:
telecom.fb_user_7.value_counts(normalize=True)*100

1.0    90.95438
0.0     9.04562
Name: fb_user_7, dtype: float64

In [17]:
telecom.fb_user_8.value_counts(normalize=True)*100

1.0    89.031873
0.0    10.968127
Name: fb_user_8, dtype: float64

### Dropping columns having more than 93% of single values


In [18]:
print("Shape before dropping 93% single value columns : ", telecom.shape)
telecom = telecom.drop(['night_pck_user_6','night_pck_user_7','night_pck_user_8'],axis=1)
print("Shape after dropping 93% single value columns : ", telecom.shape)

Shape before dropping 93% single value columns :  (69999, 159)
Shape after dropping 93% single value columns :  (69999, 156)


### Computing categorical and numeric columns

In [19]:
#Numeric
numeric_col = list(telecom.dtypes[telecom.dtypes != 'object'].index)
print((numeric_col))

['id', 'arpu_6', 'arpu_7', 'arpu_8', 'onnet_mou_6', 'onnet_mou_7', 'onnet_mou_8', 'offnet_mou_6', 'offnet_mou_7', 'offnet_mou_8', 'roam_ic_mou_6', 'roam_ic_mou_7', 'roam_ic_mou_8', 'roam_og_mou_6', 'roam_og_mou_7', 'roam_og_mou_8', 'loc_og_t2t_mou_6', 'loc_og_t2t_mou_7', 'loc_og_t2t_mou_8', 'loc_og_t2m_mou_6', 'loc_og_t2m_mou_7', 'loc_og_t2m_mou_8', 'loc_og_t2f_mou_6', 'loc_og_t2f_mou_7', 'loc_og_t2f_mou_8', 'loc_og_t2c_mou_6', 'loc_og_t2c_mou_7', 'loc_og_t2c_mou_8', 'loc_og_mou_6', 'loc_og_mou_7', 'loc_og_mou_8', 'std_og_t2t_mou_6', 'std_og_t2t_mou_7', 'std_og_t2t_mou_8', 'std_og_t2m_mou_6', 'std_og_t2m_mou_7', 'std_og_t2m_mou_8', 'std_og_t2f_mou_6', 'std_og_t2f_mou_7', 'std_og_t2f_mou_8', 'std_og_mou_6', 'std_og_mou_7', 'std_og_mou_8', 'isd_og_mou_6', 'isd_og_mou_7', 'isd_og_mou_8', 'spl_og_mou_6', 'spl_og_mou_7', 'spl_og_mou_8', 'og_others_6', 'og_others_7', 'og_others_8', 'total_og_mou_6', 'total_og_mou_7', 'total_og_mou_8', 'loc_ic_t2t_mou_6', 'loc_ic_t2t_mou_7', 'loc_ic_t2t_mou_8

In [20]:
cat_col = list(telecom.dtypes[telecom.dtypes == 'object'].index)
print((cat_col))

['date_of_last_rech_6', 'date_of_last_rech_7', 'date_of_last_rech_8', 'date_of_last_rech_data_6', 'date_of_last_rech_data_7', 'date_of_last_rech_data_8']


In [21]:
#From above data ID column from numeric type and Date columns from categorical types can be dropped
telecom = telecom.drop(cat_col,axis=1)
telecom = telecom.drop(['id'],axis=1)
print("Shape after dropping ID and date columns ", telecom.shape)


Shape after dropping ID and date columns  (69999, 149)


### Drop variables with more than a given threshold of missing values

In [22]:
# Below code gives percentage of null in every column
null_percentage = telecom.isnull().sum()/telecom.shape[0]*100

# Below code gives list of columns having more than 60% null
col_to_drop = null_percentage[null_percentage>70].keys()

intial_cols = telecom.shape[1]
telecom = telecom.drop(col_to_drop, axis=1)

In [23]:
print("Columns dropped due to 70% missing values \n")
print("\n".join(col_to_drop.to_list()))
print("\n{0} columns dropped.".format(intial_cols-telecom.shape[1]))

Columns dropped due to 70% missing values 

count_rech_2g_6
count_rech_2g_7
count_rech_2g_8
count_rech_3g_6
count_rech_3g_7
count_rech_3g_8
arpu_3g_6
arpu_3g_7
arpu_3g_8
arpu_2g_6
arpu_2g_7
arpu_2g_8
fb_user_6
fb_user_7
fb_user_8

15 columns dropped.


### Impute with mice for other missing values

In [ ]:
# using MICE technique to impute missing values in the rest of the columns
from fancyimpute import IterativeImputer as MICE
churn_imputed = MICE().fit_transform(telecom)

### By looking at data , information is spread across 3 months - June,July and August. Hence creating new DF for months

In [ ]:
def df_info(data) : 
    return pd.DataFrame({
                'Datatype' : data.dtypes.astype(str), 
                'Non_Null_Count': data.count(axis = 0).astype(int), 
                'Null_Count': data.isnull().sum().astype(int), 
                'Null_Percentage': round(data.isnull().sum()/len(data) * 100 , 2), 
                'Unique_Values_Count': data.nunique().astype(int) 
                 }).sort_values(by='Null_Percentage', ascending=False)

In [ ]:
#June month
june_columns = []
for column in telecom.columns:
    x = re.search("6$", column)
    if x:
        june_columns.append(column)
# missing_values.loc[sixth_month_columns].sort_values(by='Null_Percentage', ascending=False)
june_info = df_info(telecom)
condition = june_info.index.isin(june_columns)
june_columns = june_info[condition]
june_columns

In [ ]:
#July month
july_columns = []
for column in telecom.columns:
    x = re.search("7$", column)
    if x:
        july_columns.append(column)
# missing_values.loc[sixth_month_columns].sort_values(by='Null_Percentage', ascending=False)
july_info = df_info(telecom)
condition = june_info.index.isin(july_columns)
july_columns = july_info[condition]
july_columns

In [ ]:
#August month
aug_columns = []
for column in telecom.columns:
    x = re.search("8$", column)
    if x:
        aug_columns.append(column)
# missing_values.loc[sixth_month_columns].sort_values(by='Null_Percentage', ascending=False)
aug_info = df_info(telecom)
condition = aug_info.index.isin(aug_columns)
aug_columns = july_info[condition]
aug_columns